# Задание 4

In [ ]:
from collections.abc import Iterable
import functools

## 1. Chainer (0.4 балла)

Требуется реализовать функцию chainer, которая принимает на вход любое количество итерируемых объектов (в т.ч. контейнеров, которые тоже могут содержать итерируемые объекты) и возвращает список из всех элементов всех переданных контейнеров.

In [ ]:
def chainer(*args):
    elements = []

    if len(args) > 0:
        element = args[0]

        if (not isinstance(element, Iterable)) or (isinstance(element, str) and (len(element) == 1)):
            elements = [element, ]
        else:
            elements = chainer(*element)

        elements += chainer(*args[1:])

    return elements

Пример работы функции:

In [ ]:
chainer(1, 2, 3)

[1, 2, 3]

In [ ]:
chainer()

[]

In [ ]:
chainer([1, 2, 3], "why", ["i need", ["do that", ["?"]]])

[1,
 2,
 3,
 'w',
 'h',
 'y',
 'i',
 ' ',
 'n',
 'e',
 'e',
 'd',
 'd',
 'o',
 ' ',
 't',
 'h',
 'a',
 't',
 '?']

При этом запрещается использовать в коде конструкции циклов while и for.

## 2. Dataflow (0.6 балла)

В данном задании требуется написать обработчик генераторов, вычисляющих факториал. Для каждого генератора должна быть функция декоратор, которая инициализирует генератор.

In [ ]:
def coroutine(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        gen.send(None)
        return gen
    return wrapper

Функция генератор факториалов принимает на вход n - максимальный факториал, который может вычислить (n!).

In [ ]:
@coroutine
def fact_generator(n):
    fact = 1
    for i in range(1, n + 2):
        yield fact
        fact = fact * i

Пример:

f10 = fact_generator(10)

создает генератор, который при каждом вызове next(f10) возвращает текущее значение факториала, т. е.

next(f10) # вернет 1

next(f10) # вернет 2

next(f10) # вернет 6

...


При этом должен быть реализован функционал, который позволяет послать значение z в генератор и должно вернуться соответствующее значение факториала последовательным применением функции next. Если это невозможно (генератор исчерпался), должно быть выведено сообщение об этом и последнее возможное значение факториала.

Данный функционал должен быть реализован в классе GeneratorManeger:

In [ ]:
class GeneratorManeger(object):
    def __init__(self, *fact_desc):
        self.gens = [fact_generator(n) for n in fact_desc]
        self.max_steps = [n for n in fact_desc]
        self.cur_steps = [1] * len(fact_desc)
        self.num_active = len(fact_desc)
    
    def send(self, z):
        if self.num_active == 0:
            return 'All generators are closed'

        for i, gen in enumerate(self.gens):
            if self.cur_steps[i] <= z <= self.max_steps[i]:
                value = [next(gen) for _ in range(self.cur_steps[i], z + 1)][-1]
                
                self.cur_steps[i] = z + 1
                if self.max_steps[i] == z:
                    self.num_active -= 1

                return value

        return f"Can't calculate {z}!"

fact_desc - это список из чисел $n_i$ для каждого из которых создается свой генератор fact_generator($n_i$) и сохраняется в классе GeneratorManeger.

send должна выбрать любой генератор из еще не исчерпавшихся и вычислить значение факториала. Если таких нет, сообщить об этом.

Пример работы:

In [ ]:
gm = GeneratorManeger(3, 6)

In [ ]:
gm.send(3)

6

In [ ]:
gm.send(3)

6

In [ ]:
gm.send(3)

"Can't calculate 3!"

In [ ]:
gm.send(5)

120

In [ ]:
"Can't calculate 7!"

"Can't calculate 7!"

In [ ]:
gm.send(6)

720

In [ ]:
gm.send(1)

'All generators are closed'